In [1]:
import pandas as pd
import folium
import math

from itertools import combinations
from pyproj import Proj, transform
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

### 데이터 다운 및 전처리

In [2]:
uljin = pd.read_excel('울진군예상좌표.xlsx')
uljin = uljin[['대구분명칭', '위도', '경도']]
uljin.columns = ['이름', '위도', '경도']
uljin.head()

,이름,위도,경도
0,평해버스정류장앞 공영주차장,36.725568,129.441761
1,평해읍사무소 옆 공영주차장,36.728031,129.441030
2,북면사무소(광장) 공영주차장,37.104865,129.373088
3,죽변한전사택입구 공영주차장,37.064148,129.419173
4,구산4리(안잘미)마을주차장,36.943468,129.367039


In [3]:
market = pd.read_csv('울진_편의점.csv', header = None)
market.columns = ['경도', '위도', '이름']
market = market[['이름', '위도', '경도']]
market.head()

,이름,위도,경도
0,GS25 울진북면점,37.103837,129.374583
1,미니스톱 울진부구점,37.103012,129.374073
2,세븐일레븐 울진케이블카점,36.968704,129.401089
3,CU 북면점,37.104796,129.374379
4,미니스톱 울진북면점,37.100954,129.369387


### 위, 경도 좌표변환

In [4]:
proj_UTMK = Proj(init='epsg:5178')
proj_WGS84 = Proj(init = 'epsg:4326')

for i in  range(len(uljin)) :
    x1, y1 = transform(proj_WGS84, proj_UTMK, uljin['경도'][i], uljin['위도'][i])
    uljin['경도'][i], uljin['위도'][i] = x1, y1
    
for i in  range(len(market)) :
    x1, y1 = transform(proj_WGS84, proj_UTMK, market['경도'][i], market['위도'][i])
    market['경도'][i], market['위도'][i] = x1, y1  

### F와 L사이의 최단거리 행열 만들기

$F_i$ - uljin, 울진군 공영시설 좌표, m = 50  
$L_j$ - market, 울진군 편의점 좌표, n = 86  
$c_{ij}$ - $F_i$와 $L_j$의 최단거리

In [5]:
F_list = []
L_list = []
for i in range(len(uljin)) :
    name = 'F_{}'.format(i)
    F_list.append(name)
for i in range(len(market)) :
    name = 'L_{}'.format(i)
    L_list.append(name)

In [6]:
length_df = pd.DataFrame(columns = F_list, index = L_list)
for i in range(len(length_df)) :
    for j, col in enumerate(length_df.columns) :
        square_sum = ((uljin['위도'][j] - market['위도'][i]) ** 2) + ((uljin['경도'][j] - market['경도'][i]) ** 2)
        distance = math.sqrt(square_sum)
        length_df[col][i] = distance

### P-Median

In [7]:
char_list = []
for p in range(3, 11) :
    col_sum = list(length_df.sum(axis = 0))
    col_sum_sort = sorted(col_sum)
    min_value = col_sum_sort[:3]
    
    min_index = []
    for value in min_value :
        min_index.append(col_sum.index(value))
    
    col_sum_tuple = []
    for i in range(len(col_sum)) :
        tup = (i, col_sum[i])
        col_sum_tuple.append(tup)
    
    col_sum_tuple.sort(key = lambda x: x[1])
    col_sum_tuple = col_sum_tuple[:2*p]
    
    p_list_set = [col_sum_tuple[j:j+p] for j in range(p)]
    
    min_sum_list = []
    for k in range(len(p_list_set)) :
        tup_check = []
        for tt in p_list_set[k] :
            tup_check.append(f'F_{tt[0]}')
        check_df = length_df[tup_check]
        check_df['min'] = 0
        for l in range(len(check_df)) :
            check_df['min'][l] = min(check_df.iloc[l][:-1])
        min_sum_value = sum(check_df['min'])
        min_sum_list.append(min_sum_value)
    
    final_index = min_sum_list.index(min(min_sum_list))
    final_set = p_list_set[final_index]
    
    final_set.sort(key = lambda x : x[0])
    
    final_idx = [tp[0] for tp in final_set]
    final_uljin = uljin.iloc[final_idx, :]
    
    final_uljin.reset_index(drop = True, inplace = True)
    char_list.append(char for char in final_uljin['이름'])

In [8]:
final_kk = []
for ch in char_list :
    final_kk += ch

In [9]:
pd.Series(final_kk).value_counts()

엑스포공원 공영주차장           4
연지공원                  4
2호관사앞  공영주차장          4
연호공원 공영주차장            4
민물고기연구센터 공영주차장        4
울진종합운동장               4
성류굴종합운동장 공영주차장        3
근남농협 왕피천변 공영주차장       3
울진시장 공영주차장            3
울진등기소 앞 공영주차장         3
성류굴 공영주차장             2
남사고유적지 공영주차장          2
울진향교앞 공영주차장           2
울진군청 공영주차장            2
신성슈퍼앞(성심한의원) 공영주차장    2
구문화원 공영주차장            2
남대천변 공영주차장            2
구산4리(안잘미)마을주차장        1
망양공원                  1
dtype: int64

In [10]:
check_index = pd.Series(final_kk).value_counts().index[:6]

In [11]:
uljin_index = []
for check in check_index :
    for i in range(len(uljin)) :
        if uljin['이름'][i] == check :
            uljin_index.append(i)
            break

In [12]:
uljin_index

[16, 49, 15, 36, 29, 30]

In [13]:
uljin_sub = uljin.iloc[uljin_index, :]
uljin_sub.reset_index(drop = True, inplace = True)
uljin_sub

,이름,위도,경도
0,엑스포공원 공영주차장,1.886969e+06,1.169421e+06
1,연지공원,1.889860e+06,1.169851e+06
2,2호관사앞 공영주차장,1.890135e+06,1.169529e+06
3,연호공원 공영주차장,1.889860e+06,1.169851e+06
4,민물고기연구센터 공영주차장,1.886697e+06,1.167637e+06
5,울진종합운동장,1.889860e+06,1.169851e+06


### 시장데이터 및 최종공영시설 좌표 위경도 변환

In [14]:
uljin_sijang = pd.read_excel('울진시장.xlsx')
uljin_sijang.drop(['Unnamed: 0'], axis = 1, inplace = True)

uljin_sijang = uljin_sijang[['시장명', '위도', '경도']]
uljin_sijang

,시장명,위도,경도
0,척산 공설시장,1868071.119,1174306.722
1,척산시장,1868002.894,1174281.336
2,평해시장,1860173.407,1173628.035
3,평해 공설시장,1860147.527,1173610.691
4,매화 공설시장,1880457.937,1167725.051
5,매화시장,1880382.140,1167708.721
6,흥부시장,1901893.361,1166838.829
7,흥부 공설시장,1901893.009,1166821.055
8,죽변시장,1897023.843,1171107.094
9,죽변 공설시장,1897031.610,1171106.937


In [15]:
for i in  range(len(uljin_sijang)) :
    x1, y1 = transform(proj_UTMK, proj_WGS84, uljin_sijang['경도'][i], uljin_sijang['위도'][i])
    uljin_sijang['경도'][i], uljin_sijang['위도'][i] = x1, y1
    
for i in  range(len(uljin_sub)) :
    x1, y1 = transform(proj_UTMK, proj_WGS84, uljin_sub['경도'][i], uljin_sub['위도'][i])
    uljin_sub['경도'][i], uljin_sub['위도'][i] = x1, y1

In [16]:
m = folium.Map(location = [36.85, 129.4004195], zoom_start = 10)

In [17]:
for i in range(len(uljin_sijang)) :
    long = uljin_sijang['위도'][i]
    lat = uljin_sijang['경도'][i]
    name = uljin_sijang['시장명'][i]
    folium.Marker(location = [long, lat],
                  popup = name,
                 icon = folium.Icon(color = 'yellow',
                                    icon = 'star',
                                    size = 0.1)).add_to(m)

In [18]:
for i in range(len(uljin_sub)) :
    long = uljin_sub['위도'][i]
    lat = uljin_sub['경도'][i]
    name = uljin_sub['이름'][i]
    folium.Marker(location = [long, lat],
                  popup = name,
                 icon = folium.Icon(color = 'blue',
                                    icon = 'flag',
                                    size = 0.1)).add_to(m)

In [20]:
m